In [211]:
library(dummies)
correlates <- c("ahd","perf_status","bm_blasts","secondary","wbc","hb","plt","gender","age","os","os_status")
continuous_correlates <- c("age","wbc","hb","plt","bm_blasts","os")
df_all_components <- read.table("../../clustering/Clustering_Final_1/df_final_multiple_comp_overlap_comp.tsv")[,c(1:155,175:190)]

df_initial <- read.table("../../../data/initial_dataset/Master_04_10_2019.csv",sep = ',' , header = T)
rownames(df_initial) <- df_initial$data_pd
df_initial <- df_initial[,-1:-4]
df_eln <- read.table("../../../data/updated_dataset/eln_final.tsv",sep = '\t' , header = T)
rownames(df_eln) <- df_eln$X
df_eln <- df_eln[-1]
df <- merge(df_eln,df_all_components,by=0)
rownames(df) <- df$Row.names
df <- df[-1]
df <- merge(df,df_initial[,correlates],by=0)
rownames(df) <- df$Row.names
df <- df[-1]
for (col in colnames(df)[colSums(is.na(df)) > 0]){
    if(!is.element(col,c("os","os_status"))){        
        if(is.element(col,continuous_correlates)){
            df[,col][is.na(df[,col])] <- mean(df[,col], na.rm=TRUE)
        }else{
            df[,col][is.na(df[,col])] <- median(df[,col], na.rm=TRUE)
            }
    }
   
}
df <- na.omit(df)
df <- df[df$os>0,]

df$eln_2017 <- ifelse(df$eln_2017=="adverse",1,
                    ifelse(df$eln_2017=="intermediate",2,3))
df<- cbind(df,dummy('final_component',df, sep = "_"))
overlap_comp <- colnames(df[,startsWith(colnames(df),"overlap_"),])
df$overlap <- 0
for (comp in colnames(df[,startsWith(colnames(df),"final_component_"),])){
    df[df[,comp]==1,]$overlap <- ifelse(rowSums(df[df[,comp]==1,overlap_comp])>0,1,0)
}

df <- df[,c(1:158,176:184,187:203,160:175,185:186)]
df <- df[,colnames(df[,colSums(df)>0])]
write.table(df,"prognosis_dataframe.tsv")